I am going to design a DNN to complete the sentiment analysis task, using the popular publicly available SST2 datasets, which is a movie review dataset, each sample contains a sentence and the corresponding label of the sentence, 1 means positive and 0 means negative. for this, we need to design a suitable text embedding and DNN, such as RNN/GRU/LSTM, etc., and compare the performances of different models

In [1]:
import torch
from torch.utils.data import Dataset
import os
import pickle
import csv
from collections import Counter
import wget
from zipfile import ZipFile
from pathlib import Path

#### Text Embedding

In [2]:


def load_embedding_matrix(vocab):
    glove_file_path = "./data/glove_embeddings/glove.6B.50d.txt"
    embedding_dim = 50

    # Create a dictionary to store GloVe embeddings
    embeddings = {}

    # Read in GloVe embeddings and add them to the dictionary if they are in vocab
    with open(glove_file_path, "r", encoding='utf-8') as f:
        for line in f:
            values = line.split()
            token = values[0]
            if token in vocab:
                embedding = torch.tensor([float(val) for val in values[1:]], dtype=torch.float32)
                embeddings[token] = embedding

    # Create a tensor to store all embeddings in the dictionary
    all_embeddings = torch.stack(list(embeddings.values()), dim=0)

    # Compute the mean and standard deviation of all embeddings
    embedding_mean = torch.mean(all_embeddings)
    embedding_std = torch.std(all_embeddings)

    # Randomly initialize embeddings using the mean and standard deviation
    embedding_matrix = torch.normal(embedding_mean, embedding_std, (len(vocab), embedding_dim))

    # Replace the randomly initialized embeddings with GloVe embeddings if they exist in vocab
    for token, embedding in embeddings.items():
        embedding_matrix[vocab[token]] = embedding

    # Initialize the padding token to 0
    embedding_matrix[vocab["[pad]"]] = 0.

    return embedding_matrix


In [3]:
class SST2Dataset(Dataset):
    def __init__(self, path: Path, vocab=None, reverse_vocab=None, is_test=False):
        super().__init__()

        with open(path, "r") as f:
            reader = csv.reader(f, delimiter="\t", quoting=csv.QUOTE_NONE)
            next(reader)  # Ignore header
            data = list(reader)

        sentences = []
        labels = []

        for row in data:
            if not is_test:
                # Each row contains a sentence and label (either 0 or 1)
                sentence, label = row
                sentences.append(sentence.strip().split())
                labels.append([int(label)])
            else:
                _, sentence = row
                sentences.append(sentence.strip().split())

        self.vocab_file_path = "./data/SST-2/vocab.pkl"

        # Vocab maps tokens to indices
        if vocab is None:
            vocab = self._build_vocab(sentences)
            reverse_vocab = None

        # Reverse vocab maps indices to tokens
        if reverse_vocab is None:
            reverse_vocab = {index: token for token, index in vocab.items()}

        self.vocab = vocab
        self.reverse_vocab = reverse_vocab

        indexed_sentences = [self.tokens_to_indices(sentence) for sentence in sentences]
        if not is_test:
            labels = torch.tensor(labels)

        self.sentences = indexed_sentences
        self.labels = labels

    def __getitem__(self, index):
        return self.sentences[index], self.labels[index]

    def __len__(self):
        return len(self.sentences)

    def _build_vocab(self, sentences, unk_cutoff=1):
        # Load cached vocab if existent
        if os.path.exists(self.vocab_file_path):
            with open(self.vocab_file_path, "rb") as f:
                return pickle.load(f)

        word_counts = Counter()

        # Count unique words (lower case)
        for sentence in sentences:
            for token in sentence:
                word_counts[token.lower()] += 1

        # Special tokens: padding, beginning of sentence, end of sentence, and unknown word
        vocab = {"[pad]": 0, "[unk]": 1}
        token_id = 2

        # Assign a unique id to each word that occurs at least unk_cutoff number of times
        for token, count in word_counts.items():
            if count >= unk_cutoff:
                vocab[token] = token_id
                token_id += 1

        # Cache vocab
        with open(self.vocab_file_path, "wb") as f:
            pickle.dump(vocab, f, pickle.HIGHEST_PROTOCOL)

        return vocab

    def tokens_to_indices(self, tokens):
        """
        Converts tokens to indices.
        """
        indices = []

        unk_token = self.vocab["[unk]"]

        for token in tokens:
            indices.append(self.vocab.get(token.lower(), unk_token))

        return torch.tensor(indices)

    def indices_to_tokens(self, indices):
        """
        Converts indices to tokens and concatenates them as a string.
        """
        tokens = []

        for index in indices:
            if torch.is_tensor(index):
                index = index.item()
            token = self.reverse_vocab.get(index, "[unk]")
            if token == "[pad]":
                continue
            tokens.append(token)

        return " ".join(tokens)

In [4]:
from typing import Dict, List, Tuple
import torch
import torch.nn as nn


def collate_fn(
    batch: List[Tuple[torch.Tensor, torch.Tensor]]
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Create a batch of data given a list of N sequences and labels. Sequences are stacked into a single tensor
    of shape (N, max_sequence_length), where max_sequence_length is the maximum length of any sequence in the
    batch. Sequences shorter than this length should be filled up with 0's. Also returns a tensor of shape (N, 1)
    containing the label of each sequence.
    """
    sentences, labels = zip(*batch)
    # Get the maximum sequence length in all sentences
    max_sequence_length = max(len(seq) for seq in sentences)

    # Create a tensor to hold the padded sequences and fill it with 0
    padded_seqs = torch.zeros((len(batch), max_sequence_length))

    # Create a tensor to hold the labels and fill it with 1
    ret_labels = torch.ones((len(batch), 1))

    # Iterate over the sequences and labels in the batch
    for i, (seq, label) in enumerate(batch):
        # Copy the sequence data into the padded tensor
        padded_seqs[i, :len(seq)] = seq

        # Copy the label into the label tensor
        ret_labels[i, 0] = label

    # Return the padded sequences and labels as a tuple
    return padded_seqs, ret_labels


class RNNBinaryClassificationModel(nn.Module):
    def __init__(self, embedding_matrix: torch.Tensor, rnn_type: str):
        """Create a model with either RNN, LSTM or GRU layer followed by a linear layer.

        Args:
            embedding_matrix (torch.Tensor): Weights for embedding layer.
                Used in starter code, nothing you should worry about.
            rnn_type (str): Either "RNN", "LSTM" or "GRU". Defines what kind of a layer should be used.
        """
        super().__init__()

        vocab_size = embedding_matrix.shape[0]
        embedding_dim = embedding_matrix.shape[1]

        # Construct embedding layer and initialize with given embedding matrix. Do not modify this code.
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0
        )
        self.embedding.weight.data = embedding_matrix

        hidden_size = 64
        # Define recurrent layer
        if rnn_type == 'RNN':
            self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, batch_first=True)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, batch_first=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size=embedding_dim, hidden_size=hidden_size, batch_first=True)
        else:
            raise ValueError(f"Invalid RNN type '{rnn_type}'. Must be one of 'RNN', 'LSTM', or 'GRU'.")

        # Define output layer
        self.fc = nn.Linear(hidden_size, 1)

        # Define activation function
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """
        Takes in a batch of data of shape (N, max_sequence_length). Returns a tensor of shape (N, 1), where each
        element corresponds to the prediction for the corresponding sequence.
        """
        # convert to LongTensor
        # Embedding input sequences
        embedded = self.embedding(inputs.long())

        # Pass through recurrent layer
        output, _ = self.rnn(embedded)

        # transform the last hidden state of the recurrent layer to a shape
        logits = self.fc(output[:, -1, :])

        # Activation function
        output = self.sigmoid(logits)

        return output

    def loss(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """
        Computes the binary cross-entropy loss.
        """
        # Calculate binary cross-entropy loss
        loss_fn = nn.BCELoss()
        loss = loss_fn(logits, targets)
        return loss

    def accuracy(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """Computes the accuracy, i.e number of correct predictions / N.
        """
        # Predictions
        predictions = (logits > 0.5).float()

        # Calculate accuracy
        correct = (predictions == targets).float().sum()
        total = targets.size(0)
        accuracy = correct / total

        return accuracy

#### Define hyper-parameters

In [5]:
def get_parameters() -> Dict:
    """Returns parameters for training a model. Is have 4 entries, with these specific keys:

    {
        "TRAINING_BATCH_SIZE": TRAINING_BATCH_SIZE,  # type: int
        "VAL_BATCH_SIZE": VAL_BATCH_SIZE,  # type: int
        "NUM_EPOCHS": NUM_EPOCHS,  # type: int
        "LEARNING_RATE": LEARNING_RATE,  # type: float
    }

    Returns:
        Dict: Dictionary, as described above.
            (Feel free to copy dict above, and define TRAINING_BATCH_SIZE and LEARNING_RATE)
    """
    # Batch size for validation, this only affects performance.
    VAL_BATCH_SIZE = 128

    # Training parameters
    NUM_EPOCHS = 16
    params = {
        "TRAINING_BATCH_SIZE": 64,
        "VAL_BATCH_SIZE":VAL_BATCH_SIZE,
        "NUM_EPOCHS": NUM_EPOCHS,
        "LEARNING_RATE": 1e-4
    }
    return params

#### Load datasets

In [6]:
# Load datasets
train_dataset = SST2Dataset("./data/SST-2/train.tsv")
val_dataset = SST2Dataset("./data/SST-2/dev.tsv", train_dataset.vocab, train_dataset.reverse_vocab)

In [7]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

#### Train different models in SST-2 datasets

In [8]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


def train():
    # Get parameters from problems
    params = get_parameters()
    TRAINING_BATCH_SIZE = params["TRAINING_BATCH_SIZE"]
    NUM_EPOCHS = params["NUM_EPOCHS"]
    LEARNING_RATE = params["LEARNING_RATE"]
    VAL_BATCH_SIZE = params["VAL_BATCH_SIZE"]

    # Create data loaders for creating and iterating over batches
    train_loader = DataLoader(
        train_dataset,
        batch_size=TRAINING_BATCH_SIZE,
        collate_fn=collate_fn,
        shuffle=True,
    )
    val_loader = DataLoader(
        val_dataset, batch_size=VAL_BATCH_SIZE, collate_fn=collate_fn
    )


    # Print out some random examples from the data
    print("Data examples:")
    random_indices = torch.randperm(len(train_dataset))[:8].tolist()
    for index in random_indices:
        sequence_indices, label = (
            train_dataset.sentences[index],
            train_dataset.labels[index],
        )
        sentiment = "Positive" if label == 1 else "Negative"
        sequence = train_dataset.indices_to_tokens(sequence_indices)
        print(f"Sentiment: {sentiment}. Sentence: {sequence}")
    print()

    embedding_matrix = load_embedding_matrix(train_dataset.vocab)

    for model_type in ["LSTM", "RNN", "GRU"]:
        model = RNNBinaryClassificationModel(embedding_matrix.clone(), model_type).to(
            DEVICE
        )
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(NUM_EPOCHS):
            # Total loss across train data
            train_loss = 0.0
            # Total number of correctly predicted training labels
            train_correct = 0
            # Total number of training sequences processed
            train_seqs = 0

            print(f"Model Type: {model_type} Epoch {epoch + 1}/{NUM_EPOCHS}")
            tqdm_train_loader = tqdm(train_loader, leave=False)

            model.train()
            for batch_idx, (sentences_batch, labels_batch) in enumerate(
                tqdm_train_loader
            ):
                sentences_batch, labels_batch = (
                    sentences_batch.to(DEVICE),
                    labels_batch.to(DEVICE),
                )

                # Make predictions
                logits = model(sentences_batch)

                # Compute loss and number of correct predictions
                loss = model.loss(logits, labels_batch)
                correct = model.accuracy(logits, labels_batch).item() * len(logits)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Accumulate metrics and update status
                train_loss += loss.item()
                train_correct += correct
                train_seqs += len(sentences_batch)
                tqdm_train_loader.set_description_str(
                    f"[Loss]: {train_loss / (batch_idx + 1):.4f} [Acc]: {train_correct / train_seqs:.4f}"
                )
            tqdm_train_loader.close()
            print()

            avg_train_loss = train_loss / len(tqdm_train_loader)
            train_accuracy = train_correct / train_seqs
            print(
                f"\t[Training Loss]: {avg_train_loss:.4f} [Training Accuracy]: {train_accuracy:.4f}"
            )

            # Total loss across validation data
            val_loss = 0.0
            # Total number of correctly predicted validation labels
            val_correct = 0
            # Total number of validation sequences processed
            val_seqs = 0

            tqdm_val_loader = tqdm(val_loader, leave=False)

            model.eval()
            for batch_idx, (sentences_batch, labels_batch) in enumerate(
                tqdm_val_loader
            ):
                sentences_batch, labels_batch = (
                    sentences_batch.to(DEVICE),
                    labels_batch.to(DEVICE),
                )

                with torch.no_grad():
                    # Make predictions
                    logits = model(sentences_batch)

                    # Compute loss and number of correct predictions and accumulate metrics and update status
                    val_loss += model.loss(logits, labels_batch).item()
                    val_correct += model.accuracy(logits, labels_batch).item() * len(
                        logits
                    )
                    val_seqs += len(sentences_batch)
                    tqdm_val_loader.set_description_str(
                        f"[Loss]: {val_loss / (batch_idx + 1):.4f} [Acc]: {val_correct / val_seqs:.4f}"
                    )
            tqdm_val_loader.close()
            print()

            avg_val_loss = val_loss / len(tqdm_val_loader)
            val_accuracy = val_correct / val_seqs
            print(
                f"\t[Validation Loss]: {avg_val_loss:.4f} [Validation Accuracy]: {val_accuracy:.4f}"
            )

        # Due to the lack of labels in the SST-2 test set, it is not possible to predict test accuracy


In [9]:
train()

Data examples:
Sentiment: Negative. Sentence: hell-bent
Sentiment: Positive. Sentence: most entertaining moments
Sentiment: Positive. Sentence: rising above similar fare
Sentiment: Positive. Sentence: raunchy and frequently hilarious
Sentiment: Negative. Sentence: as a seven rip-off
Sentiment: Negative. Sentence: chafing
Sentiment: Negative. Sentence: fails to portray its literarily talented and notorious subject as anything much more than a dirty old man .
Sentiment: Positive. Sentence: enter and accept another world

Model Type: LSTM Epoch 1/16



	[Training Loss]: 0.6869 [Training Accuracy]: 0.5569



	[Validation Loss]: 0.6988 [Validation Accuracy]: 0.5103
Model Type: LSTM Epoch 2/16



	[Training Loss]: 0.5649 [Training Accuracy]: 0.7096



	[Validation Loss]: 0.5942 [Validation Accuracy]: 0.7397
Model Type: LSTM Epoch 3/16


...


	[Validation Loss]: 0.6348 [Validation Accuracy]: 0.8016
Model Type: GRU Epoch 15/16



	[Training Loss]: 0.1488 [Training Accuracy]: 0.9476



	[Validation Loss]: 0.6184 [Validation Accuracy]: 0.8039
Model Type: GRU Epoch 16/16



	[Training Loss]: 0.1415 [Training Accuracy]: 0.9498



	[Validation Loss]: 0.7153 [Validation Accuracy]: 0.8108


Due to the lack of labels in the SST-2 test set, it is not possible to predict test accuracy.
The best validation accuracy in different models as follow:

| Model | Best Validation Accuracy |
| :---: |:------------------------:|
|  RNN  |          68.81%          |
|  GRU  |          81.08%          |
| LSTM  |          80.50%          |